In [ ]:
#Farklı metodla SVM:

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Excel dosyasından verileri oku
df = pd.read_excel("/content/covidsonrasi.xlsx")

# NaN değerlere sahip satırları sil
df = df.dropna()

sentences = df["tweets"].values  # Cümleler
labels = df["total"].values  # Sınıf etiketleri

# Özellik çıkarma: cümleleri kelime sayısına göre özellik olarak temsil et
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['tweets'].values.astype(str))

# Özellikleri standartlaştır
scaler = StandardScaler(with_mean=False, with_std=False)
X_scaled = scaler.fit_transform(X)

# Veri kümesini eğitim ve test kümelerine ayır
X_train, X_test, y_train, y_test = train_test_split(X_scaled, labels, test_size=0.3, random_state=1)

# SVM modelini eğit
svm_clf = LinearSVC(C=1, loss="hinge")
svm_clf.fit(X_train, y_train)

# Test kümesini kullanarak modeli değerlendir
y_pred = svm_clf.predict(X_test)

# Doğruluk (Accuracy) hesapla
accuracy = accuracy_score(y_test, y_pred)

# Hassasiyet (Precision) hesapla (Makro ve Mikro)
precision_macro = precision_score(y_test, y_pred, average='macro')
precision_micro = precision_score(y_test, y_pred, average='micro')

# Geri çağırma (Recall) hesapla (Makro ve Mikro)
recall_macro = recall_score(y_test, y_pred, average='macro')
recall_micro = recall_score(y_test, y_pred, average='micro')

# F1 skoru hesapla (Makro ve Mikro)
f1_macro = f1_score(y_test, y_pred, average='macro')
f1_micro = f1_score(y_test, y_pred, average='micro')

print("Doğruluk (Accuracy): {:.4f}".format(accuracy))
print("Hassasiyet (Precision Macro): {:.4f}".format(precision_macro))
print("Hassasiyet (Precision Micro): {:.4f}".format(precision_micro))
print("Geri Çağırma (Recall Macro): {:.4f}".format(recall_macro))
print("Geri Çağırma (Recall Micro): {:.4f}".format(recall_micro))
print("F1 Skoru (Macro): {:.4f}".format(f1_macro))
print("F1 Skoru (Micro): {:.4f}".format(f1_micro))




Doğruluk (Accuracy): 0.6388
Hassasiyet (Precision Macro): 0.5876
Hassasiyet (Precision Micro): 0.6388
Geri Çağırma (Recall Macro): 0.5787
Geri Çağırma (Recall Micro): 0.6388
F1 Skoru (Macro): 0.5804
F1 Skoru (Micro): 0.6388


In [ ]:
from sklearn.metrics import classification_report

# Predict labels for the test set
y_pred = svm_clf.predict(X_test)

# Calculate and print the classification report
report = classification_report(y_test, y_pred, output_dict=True)

# Print the modified report
print("{:<10} {:<10} {:<10} {:<10} {:<10}".format("Class", "Precision", "Recall", "F1-Score", "Support"))
for label, metrics in report.items():
    if label != "accuracy":
        precision = metrics.get('precision', {})
        recall = metrics.get('recall', {})
        f1_score = metrics.get('f1-score', {})
        support = metrics.get('support', {})

        precision_str = "{:.2f}%".format(precision * 100) if isinstance(precision, float) else "N/A"
        recall_str = "{:.2f}%".format(recall * 100) if isinstance(recall, float) else "N/A"
        f1_score_str = "{:.2f}%".format(f1_score * 100) if isinstance(f1_score, float) else "N/A"
        support_str = str(support) if isinstance(support, int) else "N/A"

        print("{:<10} {:<10} {:<10} {:<10} {:<10}".format(label, precision_str, recall_str, f1_score_str, support_str))


Class      Precision  Recall     F1-Score   Support   
0          69.79%     75.71%     72.63%     1684      
1          45.54%     38.26%     41.58%     894       
macro avg  57.66%     56.98%     57.10%     2578      
weighted avg 61.38%     62.72%     61.86%     2578      


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# df, vectorizer, scaler gibi değişkenlerin ve X_train, X_test, y_train, y_test'in tanımlandığı bir yer olduğunu varsayalım

# Binarize the labels
y_test_bin = label_binarize(y_test, classes=[0, 1, 2])

# Train the Naive Bayes classifier
nb_clf = MultinomialNB()
nb_clf.fit(X_train, y_train)

# Calculate the class probabilities
y_pred_prob = nb_clf.predict_proba(X_test)

# Calculate the false positive rate, true positive rate, and AUC for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
num_classes = 3  # Sınıf sayısı
for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_pred_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot the ROC curves for each class
plt.figure()
colors = ['blue', 'red', 'green']
for i, color in zip(range(num_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2, label='Class {} (AUC = {:.2f})'.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - OvA')
plt.legend(loc="lower right")
plt.show()


IndexError: ignored

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score

# Determine the number of classes
num_classes = len(np.unique(y_test))

# Calculate the ROC AUC score for each class
roc_area = []
for i in range(num_classes):
    roc_area.append(round(roc_auc_score(y_test_bin[:, i], y_pred_prob[:, i]), 2))

print("Approximate ROC Area for each class:", roc_area)


IndexError: ignored

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# y_test ve y_pred gibi değişkenlerin tanımlandığı bir yer olduğunu varsayalım
y_pred = svm_clf.predict(X_test)
# Calculate precision, recall, and F-measure
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f_measure = f1_score(y_test, y_pred, average='macro')

print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F-measure: {:.2f}".format(f_measure))



Precision: 0.66
Recall: 0.64
F-measure: 0.64


In [ ]:
# Calculate RMSE
from sklearn.metrics import mean_squared_error

import numpy as np
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: {:.2f}".format(rmse))


RMSE: 0.50


In [ ]:
from sklearn.metrics import recall_score

# y_pred ve y_test'in tanımlandığı bir yer olduğunu varsayalım

# Calculate recall
recall = recall_score(y_test, y_pred, average='macro')
print("Recall: {:.2f}".format(recall))


Recall: 0.64


In [ ]:
from sklearn.metrics import precision_score

# y_pred ve y_test'in tanımlandığı bir yer olduğunu varsayalım

# Calculate precision
precision = precision_score(y_test, y_pred, average='macro')
print("Precision: {:.2f}".format(precision))


Precision: 0.66
